In [54]:
# api 사용하기 위한 라이브러리 설치
!pip install google-api-python-client

In [171]:
# 라이브러러 로드
import pandas as pd
import requests
from googleapiclient.discovery import build

# 시간 라이브러리
from tqdm import tqdm, trange
import time

# 경고창 무시
import warnings
warnings.filterwarnings('ignore')

In [205]:
api_key = 'api_key'
youtube = build('youtube', 'v3', developerKey=api_key)

In [207]:
input_url = input("채널 URL을 입력해주세요 (https:// 제외) : ")

채널 URL을 입력해주세요 (https:// 제외) : www.youtube.com/c/슈카월드


In [208]:
channel_name = input_url.split('/')[-1]
channel_name

'슈카월드'

In [209]:
request = youtube.search().list( 
    q=channel_name, 
    order='relevance',
    part="snippet", 
    maxResults=50
    ).execute()
channel_id = request['items'][0]['id']['channelId']
channel_id

'UCsJ6RuBiTVWRX156FVbeaGg'

In [157]:
playlists = youtube.playlists().list(channelId=channel_id,
                                    part="snippet",
                                    maxResults=20).execute()

In [162]:
playlist_title = []
playlist_id = []

for data in playlists['items']:
    playlist_title.append(data['snippet']['title'])
    playlist_id.append(data['id'])

In [165]:
df = pd.DataFrame(columns=["playlist", "playlist_id"])

In [166]:
df['playlist'] = playlist_title
df['playlist_id'] = playlist_id
df

,playlist,playlist_id
0,명예의 전당 (역대 금주의 추천 영상들),PLJPjg3It2DXTinXdd8uGHCk9FReAe21no
1,금주의 추천 영상,PLJPjg3It2DXRLfIh4oG4tA2Z0hBAP6K3t
2,연휴 특집,PLJPjg3It2DXRkQFl2XeXt6kWjZBdVlnEj
3,꿀잼 세계사,PLJPjg3It2DXSmRlcwlsP3sf2RDt1EX4Dp
4,Trading Idea를 찾아서,PLJPjg3It2DXTjyMDpv3529npTRIfynn_P
5,슈카's 추천영상,PLJPjg3It2DXQ2lPgdpD7bcKo-baK4lUpW
6,방구석 전문가의 이불밖 탐험,PLJPjg3It2DXRaWcHvoZ-rqUsLcXPlH77O
7,이슈&뉴스,PLJPjg3It2DXSAD6OzQ7QON6li-QcbBFOl
8,레전(前)썰,PLJPjg3It2DXQxAtY-8bViwEe9aSRCHfds
9,슈카의 일상썰,PLJPjg3It2DXTKV8xwYQoQsRM3IlwQN4Kp


In [169]:
# 재생목록에서 동영상들 정보 입력

video_ids = []
video_titles = []
video_dates = []

for playlistId in df['playlist_id']:
    playlist_videos = youtube.playlistItems().list(playlistId=playlistId,
                                                  part="snippet",
                                                  maxResults=50)
    while playlist_videos:
        playlistitems_list_response = playlist_videos.execute()
        
        for playlist_item in playlistitems_list_response['items']:
            title = playlist_item["snippet"]["title"]
            video_id = playlist_item["snippet"]["resourceId"]["videoId"]
            video_ids.append(video_id)
            video_titles.append(title)
        
        playlist_videos = youtube.playlistItems().list_next(playlist_videos,
                                                           playlistitems_list_response)
    

video_df = pd.DataFrame()
video_df['title'] = video_titles
video_df['video id'] = video_ids

In [170]:
video_df

,title,video id
0,"10년의 치킨게임, 최악의 선택, 그리고 한진해운의 파산",Wx-wwcd9XvQ
1,집값을 20배 올려주는 마법의 낙서,rKqlP-FvEas
2,"너무 힘쎈 이웃 러시아, 쪼개질 위험에 처한 눈물의 우크라이나",Gj3oo-hZTk8
3,금리에 대해 오해하고 있는 것들 (초등학생도 이해 가능한 ver.),k6i_4oGdx_E
4,"미국 최연소 연방거래 위원장 32세 리나 칸, '아마존 킬러'의 등장",zchT8jvFAV8
...,...,...
1263,식빵+슈카 아재토크 6회 공무원이 선망받는 직업이 된 이유,KNlURTebh3w
1264,식빵+슈카 아재토크 7회 상상초월 주총꾼 썰,LHNRb2HuJs8
1265,식빵+슈카 아재토크 8회 출자전환의 숨은 뜻,PPfClfv2HaA
1266,식빵+슈카아재토크 9화 외환위기 IMF의 추억,9D_lAtd1WRw


In [194]:
title = []
ids = []
views = []
likes = []
comments = []
mins = []

for i in trange(len(video_df)):
    request = youtube.videos().list(
        part="snippet, contentDetails, statistics",
        id=video_df['video id'][i])
    response = request.execute()
    
    if response['items']:
        title.append(response['items'][0]['snippet']['title'])
        ids.append(video_df['video id'][i])
        views.append(response['items'][0]['statistics']['viewCount'])
        likes.append(response['items'][0]['statistics']['likeCount'])
        comments.append(response['items'][0]['statistics']['commentCount'])
    else:
        title.append(None)
        ids.append(None)
        views.append(None)
        likes.append(None)
        comments.append(None)
        mins.append(None)
        
        time.sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████| 1268/1268 [02:17<00:00,  9.24it/s]


In [195]:
final_df = pd.DataFrame(columns=['title', 'ids', 'views', 'likes', 'comments'])

In [196]:
final_df['title'] = title
final_df['ids'] = ids
final_df['views'] = views
final_df['likes'] = likes
final_df['comments'] = comments

In [204]:
# 중복 제거 후 index reset
final_df = final_df.drop_duplicates()
final_df = final_df.reset_index(drop=True)
final_df

,title,ids,views,likes,comments
0,"10년의 치킨게임, 최악의 선택, 그리고 한진해운의 파산",Wx-wwcd9XvQ,323973,4887,396
1,집값을 20배 올려주는 마법의 낙서,rKqlP-FvEas,279244,4059,223
2,"너무 힘쎈 이웃 러시아, 쪼개질 위험에 처한 눈물의 우크라이나",Gj3oo-hZTk8,3741460,44378,2154
3,금리에 대해 오해하고 있는 것들 (초등학생도 이해 가능한 ver.),k6i_4oGdx_E,735723,14077,616
4,"미국 최연소 연방거래 위원장 32세 리나 칸, '아마존 킬러'의 등장",zchT8jvFAV8,403681,7394,1186
...,...,...,...,...,...
1143,식빵+슈카 아재토크 6회 공무원이 선망받는 직업이 된 이유,KNlURTebh3w,29903,363,26
1144,식빵+슈카 아재토크 7회 상상초월 주총꾼 썰,LHNRb2HuJs8,70563,673,48
1145,식빵+슈카 아재토크 8회 출자전환의 숨은 뜻,PPfClfv2HaA,107564,555,61
1146,식빵+슈카아재토크 9화 외환위기 IMF의 추억,9D_lAtd1WRw,35977,439,40


## 함수 버전

In [210]:
def get_summary_videos(input_url):
    # channel_id 얻기 위해 URL split
    channel_name = input_url.split('/')[-1]

    # channel_id가 다른 도메인으로 설정되어있는 경우 api를 이용해 수집
    request = youtube.search().list(q=channel_name, order='relevance', 
                                    part="snippet", maxResults=50).execute()
    channel_id = request['items'][0]['id']['channelId']

    # 채널에 있는 재생목록 가져오기
    playlists = youtube.playlists().list(channelId=channel_id, part="snippet", 
                                         maxResults=20).execute()

    # 재생목록 이름과 id 수집
    playlist_title = []
    playlist_id = []

    for data in playlists['items']:
        playlist_title.append(data['snippet']['title'])
        playlist_id.append(data['id'])

        time.sleep(0.1)
    
    # 재생목록 df로 만들기
    playlist_df = pd.DataFrame(columns=["playlist", "playlist_id"])
    playlist_df['playlist'] = playlist_title
    playlist_df['playlist_id'] = playlist_id

    # 재생목록 id와 api를 이용하여 재생목록 안에 있는 동영상들 제목과 id 수집
    video_ids = []
    video_titles = []
    video_dates = []

    for playlistId in playlist_df['playlist_id']:
        videos = youtube.playlistItems().list(playlistId=playlistId, 
                                              part="snippet",
                                              maxResults=50)
        while videos:
            videos_response = videos.execute()
            
            for playlist_item in videos_response['items']:
                title = playlist_item["snippet"]["title"]
                video_id = playlist_item["snippet"]["resourceId"]["videoId"]
                video_ids.append(video_id)
                video_titles.append(title)
            
            videos = youtube.playlistItems().list_next(videos, 
                                                       videos_response)
        
        time.sleep(0.1)

    # 동영상 df로 만들기
    video_df = pd.DataFrame()
    video_df['title'] = video_titles
    video_df['video id'] = video_ids

    # 동영상 df에서 id와 api를 이용하여 데이터 수집
    # 제목, video_id, 조회수, 좋아요 수, 댓글 수 수집
    title = []
    ids = []
    views = []
    likes = []
    comments = []

    for i in trange(len(video_df)):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=video_df['video id'][i])
        response = request.execute()
        
        if response['items']:
            title.append(response['items'][0]['snippet']['title'])
            ids.append(video_df['video id'][i])
            views.append(response['items'][0]['statistics']['viewCount'])
            likes.append(response['items'][0]['statistics']['likeCount'])
            comments.append(response['items'][0]['statistics']['commentCount'])
        else:
            title.append(None)
            ids.append(None)
            views.append(None)
            likes.append(None)
            comments.append(None)
            
            time.sleep(0.1)

    # 수집한 데이터 df
    df = pd.DataFrame(columns=['title', 'ids', 'views', 'likes', 'comments'])
    df['title'] = title
    df['ids'] = ids
    df['views'] = views
    df['likes'] = likes
    df['comments'] = comments

    # 중복 제거 후 index reset
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)

    return df

In [212]:
input_url = input("채널 URL을 입력해주세요 (https:// 제외) : ")

get_summary_videos(input_url)

채널 URL을 입력해주세요 (https:// 제외) : www.youtube.com/c/파라과이맨


100%|████████████████████████████████████████████████████████████████████████████| 80/80 [00:10<00:00,  7.48it/s]


,title,ids,views,likes,comments
0,이제는 가난하지 않은 미국 필라델피아 의사 브이로그,ODxLX5MVpCw,777914,10742,768
1,미국 전문의 월세 230만원 필라델피아 아파트,1lT6tojJMmY,93075,2293,214
2,남미에서 밤 무대 뛰는 의사,PYNAVAY98ro,31142,1441,87
3,어머니의 파라과이 옷가게,3Ec7yyiQbCU,73252,2632,168
4,내 고향 파라과이 (아순시온),dh_1eZ3tVZ0,57808,3100,190
5,미국 보스턴 펠로우 주말 당직 생존기,7OdFrabSFis,52669,1543,147
6,미국 보스턴 새 집! 월세가 얼마일까요? 누가 뉴욕보다 싸다고 했음?,wUoa593Ad6c,100597,2536,194
7,해외 미국 바이올린 버스킹 한인 의사,XmTDCdA39dQ,31849,2064,147
8,가난한 미국 의사가 월세 내는 법,qFHicIpjKiw,95120,2933,235
9,가난한 보스턴 미국의사의 소박한 일상,5Tz3H_X5mlc,104046,3094,270
